# Exercise 05 : Pandas optimizations

## Imports

In [33]:
import pandas as pd
import gc

## Read CSV file

In [34]:
df = pd.read_csv('../data/fines.csv')
df

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.0,Ford,Focus,1989
1,E432XX77RUS,1,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1,2100.0,Ford,Focus,1984
3,X582HE161RUS,2,2000.0,Ford,Focus,2015
4,92918M178RUS,1,5700.0,Ford,Focus,2014
...,...,...,...,...,...,...
925,SCHOOL21000RUS,2,500.0,Ford,Focus,2000
926,SCHOOL21001RUS,1,18500.0,Volkswagen,Focus,1996
927,SCHOOL21002RUS,2,3800.0,Ford,Focus,2013
928,SCHOOL21003RUS,2,1600.0,Ford,Focus,1992


## Iterations: loop

In [35]:
def iterations_loop(df_loop):
    result = []
    for i in range(0, len(df_loop)):
        result.append(df_loop.iloc[i]['Fines'] / df_loop.iloc[i]['Refund'] * df_loop.iloc[i]['Year'])
    df_loop['Calculated'] = result

In [36]:
%%timeit

df['Sum1'] = iterations_loop(df)

215 ms ± 6.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Iterations: *iterrows()*

In [37]:
def iterations_iterrows(df_rows):
    result = []
    for row in df_rows.iterrows():
        result.append(row[1]['Fines'] / row[1]['Refund'] * row[1]['Year'])
    df_rows['Calculated'] = result

In [38]:
%%timeit

df['Sum2'] = iterations_iterrows(df)

42.5 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Iterations: *apply()*

In [39]:
def iterations_apply(df_apply):
    df_apply['Calculated'] = df_apply.apply(lambda row: row['Fines'] / row['Refund'] * row['Year'], axis='columns')

In [40]:
%%timeit

df['Sum3'] = iterations_apply(df)

10.3 ms ± 58.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Iterations: Series

In [41]:
def iterations_series(df_series):
    df_series['Calculated'] = df_series['Fines'] / df_series['Refund'] * df_series['Year']

In [42]:
%%timeit

df['Sum4'] = iterations_series(df)

344 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Iterations: Series and values

In [43]:
def iterations_series_values(df_series_values):
    df_series_values['Calculated'] = df_series_values['Fines'].values / df_series_values['Refund'].values * df_series_values['Year'].values

In [44]:
%%timeit

df['Sum5'] = iterations_series_values(df)

166 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Index

In [45]:
%%timeit

df[df['CarNumber'] == 'O136HO197RUS']

315 µs ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
df.set_index('CarNumber', inplace=True)

In [47]:
%%timeit

df.loc['O136HO197RUS']

149 µs ± 758 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Downcasting

In [48]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to SCHOOL21004RUS
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Refund      930 non-null    int64  
 1   Fines       930 non-null    float64
 2   Make        930 non-null    object 
 3   Model       919 non-null    object 
 4   Year        930 non-null    int64  
 5   Calculated  930 non-null    float64
 6   Sum1        0 non-null      object 
 7   Sum2        0 non-null      object 
 8   Sum3        0 non-null      object 
 9   Sum4        0 non-null      object 
 10  Sum5        0 non-null      object 
dtypes: float64(2), int64(2), object(7)
memory usage: 345.0 KB


In [49]:
copy = df.copy()

In [50]:
fcols = copy.select_dtypes('float').columns
icols = copy.select_dtypes('integer').columns

copy[fcols] = copy[fcols].apply(pd.to_numeric, downcast='float')
copy[icols] = copy[icols].apply(pd.to_numeric, downcast='integer')

In [51]:
copy.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to SCHOOL21004RUS
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Refund      930 non-null    int8   
 1   Fines       930 non-null    float32
 2   Make        930 non-null    object 
 3   Model       919 non-null    object 
 4   Year        930 non-null    int16  
 5   Calculated  930 non-null    float64
 6   Sum1        0 non-null      object 
 7   Sum2        0 non-null      object 
 8   Sum3        0 non-null      object 
 9   Sum4        0 non-null      object 
 10  Sum5        0 non-null      object 
dtypes: float32(1), float64(1), int16(1), int8(1), object(7)
memory usage: 329.5 KB


## Categories

In [52]:
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [53]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to SCHOOL21004RUS
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Refund      930 non-null    int64   
 1   Fines       930 non-null    float64 
 2   Make        930 non-null    category
 3   Model       919 non-null    category
 4   Year        930 non-null    int64   
 5   Calculated  930 non-null    float64 
 6   Sum1        0 non-null      category
 7   Sum2        0 non-null      category
 8   Sum3        0 non-null      category
 9   Sum4        0 non-null      category
 10  Sum5        0 non-null      category
dtypes: category(7), float64(2), int64(2)
memory usage: 132.3 KB


##  Memory clean

In [54]:
%reset_selective df

gc.collect()

7